In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
import shutil
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os
import time
from tensorflow.keras import datasets, layers, models, losses
from tensorflow.keras.callbacks import EarlyStopping
import random
from keras.optimizers import SGD
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, BatchNormalization

In [2]:
IMG_SIZE=256
s = []
labels = []

files_list = os.listdir(r"C:\Users\shriy\OneDrive\Documents\WCEBleedGen\bleeding\Images")
for file_name in files_list:
    path=(r"C:\Users\shriy\OneDrive\Documents\WCEBleedGen\bleeding\Images"+"/"+file_name)
    image=cv2.resize(cv2.imread(path),(IMG_SIZE,IMG_SIZE))  
    s.append(np.array(image))  
    labels.append(1)
    
files_list2 = os.listdir(r"C:\Users\shriy\OneDrive\Documents\WCEBleedGen\non-bleeding\images")
for file_name in files_list2:
    path=(r"C:\Users\shriy\OneDrive\Documents\WCEBleedGen\non-bleeding\images"+"/"+file_name)
    image=cv2.resize(cv2.imread(path),(IMG_SIZE,IMG_SIZE))  
    s.append(np.array(image))  
    labels.append(0)
features = np.asarray(s)
labels = np.array(labels)

In [3]:
seed_constant=27
np.random.seed(seed_constant)
random.seed(seed_constant)
tf.random.set_seed(seed_constant)
features_train, features_test, labels_train, labels_test = train_test_split(features, labels,
                                                                            test_size = 0.2, shuffle = True,
                                                                            random_state=seed_constant
                                                                            )

In [4]:
cnn = tf.keras.Sequential(
    [
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation="relu",input_shape=(256, 256, 3)),
    BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Dropout(0.10),

    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation="relu"),
    BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Dropout(0.10),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Dropout(0.10),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(512,activation='relu'),
    tf.keras.layers.Dropout(0.50),
    tf.keras.layers.Dense(254,activation='relu'),
    tf.keras.layers.Dropout(0.50),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dropout(0.10),
    tf.keras.layers.Dense(64,activation='relu'),
    tf.keras.layers.Dropout(0.10),
    tf.keras.layers.Dense(1, activation="sigmoid")
]
)

In [5]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 batch_normalization (BatchN  (None, 256, 256, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 128, 128, 32)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 128, 128, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 64)      18496     
                                                                 
 batch_normalization_1 (Batc  (None, 128, 128, 64)     2

In [6]:
early_stopping_callback = EarlyStopping(monitor = 'val_accuracy', patience = 5, mode = 'max', restore_best_weights = True)

cnn.compile(optimizer="adam",
             loss='binary_crossentropy',
             metrics=['accuracy'])

In [7]:
history =cnn.fit(features_train,labels_train,epochs=15, batch_size = 32 ,shuffle = True,validation_split = 0.20,callbacks=[early_stopping_callback])

Epoch 1/15
53/53 [==============================] - 272s 5s/step - loss: 1.2319 - accuracy: 0.6430 - val_loss: 0.7058 - val_accuracy: 0.6110
Epoch 2/15
53/53 [==============================] - 244s 5s/step - loss: 0.2261 - accuracy: 0.9200 - val_loss: 0.3439 - val_accuracy: 0.8496
Epoch 3/15
53/53 [==============================] - 267s 5s/step - loss: 0.1084 - accuracy: 0.9630 - val_loss: 0.5397 - val_accuracy: 0.7184
Epoch 4/15
53/53 [==============================] - 249s 5s/step - loss: 0.0582 - accuracy: 0.9803 - val_loss: 0.1459 - val_accuracy: 0.9427
Epoch 5/15
53/53 [==============================] - 257s 5s/step - loss: 0.0409 - accuracy: 0.9845 - val_loss: 0.0998 - val_accuracy: 0.9499
Epoch 6/15
53/53 [==============================] - 259s 5s/step - loss: 0.0490 - accuracy: 0.9857 - val_loss: 0.2729 - val_accuracy: 0.9236
Epoch 7/15
53/53 [==============================] - 245s 5s/step - loss: 0.1124 - accuracy: 0.9672 - val_loss: 0.1402 - val_accuracy: 0.9523
Epoch 8/15
53

In [38]:
im=[]
files_list = os.listdir(r"C:\Users\shriy\OneDrive\Documents\Auto-WCEBleedGen Challenge Test Dataset\Test Dataset 2")
for file_name in files_list:
    path=(r"C:\Users\shriy\OneDrive\Documents\Auto-WCEBleedGen Challenge Test Dataset\Test Dataset 2"+"/"+file_name)
    image=cv2.resize(cv2.imread(path),(IMG_SIZE,IMG_SIZE))  
    im.append(np.array(image))
x_test=np.asarray(im)

In [114]:
l=[]
for i in files_list:
    l.append(i[0:-4])


In [18]:

y_pred=cnn.predict(x_test)


17/17 [==============================] - 13s 740ms/step


In [41]:
y_pred_classes=[]
for i in y_pred:
    if i>0.5:
        y_pred_classes.append('bleeding')
    else:
        y_pred_classes.append('non-bleeding')


In [8]:
accuracy = cnn.evaluate(features_test, labels_test)
print(accuracy)

17/17 [==============================] - 12s 711ms/step - loss: 0.2892 - accuracy: 0.9809
[0.289240300655365, 0.9809160232543945]


In [34]:
confusion_matrix(labels_test, Y_pred_classes)

array([[245,   0],
       [ 10, 269]], dtype=int64)

In [113]:
from sklearn.metrics import recall_score

# Calculating the recall score of classifier
print(f"Recall Score of the classifier is: {recall_score(labels_test,Y_pred_classes)}")
from sklearn.metrics import f1_score

# Calculating the F1 score of classifier
print(f"F1 Score of the classifier is: {f1_score(labels_test, Y_pred_classes)}")

Recall Score of the classifier is: 0.96415770609319
F1 Score of the classifier is: 0.9817518248175183


In [49]:
df = pd.DataFrame(list(zip(l, y_pred_classes)),
               columns =['Image_name', 'Predicted Class label'])
df

,Image_name,Predicted Class label
0,A0050,bleeding
1,A0051,non-bleeding
2,A0052,bleeding
3,A0053,non-bleeding
4,A0054,non-bleeding
...,...,...
510,A0560,bleeding
511,A0561,non-bleeding
512,A0562,non-bleeding
513,A0563,non-bleeding


In [50]:
df.to_csv('submission.csv',index=False)

In [64]:
im1=[]
files_list1 = os.listdir(r"C:\Users\shriy\OneDrive\Documents\Auto-WCEBleedGen Challenge Test Dataset\Test Dataset 1")
for file_name in files_list1:
    path1=(r"C:\Users\shriy\OneDrive\Documents\Auto-WCEBleedGen Challenge Test Dataset\Test Dataset 1"+"/"+file_name)
    image1=cv2.resize(cv2.imread(path1),(IMG_SIZE,IMG_SIZE))  
    im1.append(np.array(image1))
x1_test=np.asarray(im1)

In [65]:
l1=[]
for i in files_list1:
    l1.append(i[0:-4])

In [66]:
y1_pred=cnn.predict(x1_test)

2/2 [==============================] - 2s 527ms/step


In [67]:
y1_pred_classes=[]
for i in y1_pred:
    if i>0.5:
        y1_pred_classes.append('bleeding')
    else:
        y1_pred_classes.append('non-bleeding')


In [70]:
df1 = pd.DataFrame(list(zip(l1, y1_pred_classes)),
               columns =['Image_name', 'Predicted Class label'])
df1

,Image_name,Predicted Class label
0,A0000,bleeding
1,A0001,non-bleeding
2,A0002,bleeding
3,A0003,non-bleeding
4,A0004,non-bleeding
5,A0005,non-bleeding
6,A0006,non-bleeding
7,A0007,non-bleeding
8,A0008,non-bleeding
9,A0009,non-bleeding


In [111]:
# read label
labels = pd.read_csv(r"C:\Users\shriy\Downloads\submission.csv")
#labels
# sort label
labels = labels.sort_values(by=['Predicted Class label'])
labels=labels[:252]
# take unique labels
class_names = list(labels['Image_name'])

# loc of train data
#train_images = r'C:\Users\shriy\OneDrive\Documents\Auto-WCEBleedGen Challenge Test Dataset\Test Dataset 2'
# loc of labels with img
#train_cat = r'C:\Users\shriy\Downloads\data_to_predict'
#creating subfolders
for c in class_names:
    dest = 'C:/Users/shriy/Downloads/data_to_predict/' 
    for i in class_names: # Image Id
        get_image = os.path.join('C:/Users/shriy/OneDrive/Documents/Auto-WCEBleedGen Challenge Test Dataset/Test Dataset 2/'+ i +'.png') # Path to Images 
        move_image_to_cat = shutil.copy(get_image, dest)